Load libraries

In [1]:
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, IDF, StringIndexer, PCA, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

from pyspark.sql.functions import udf, avg, col
from pyspark.sql.types import IntegerType

import re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1623841700318_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Load and persist data

In [2]:
stack_overflow_data = "s3n://sparkify-mr/data/Train_onetag_small.json"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df2 = spark.read.json(stack_overflow_data)
df2.persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

Define pipeline

In [4]:
# The next three create the features
# Tokenizer
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
# Vectorizer
cv = CountVectorizer(inputCol="words", outputCol="TF", vocabSize=10000)
# IDF Transformer
idf = IDF(inputCol="TF", outputCol="features")

# This creates the label
# Indexer
indexer = StringIndexer(inputCol="oneTag", outputCol="label")

# Logistic regression model
lr =  LogisticRegression(maxIter=10, regParam=0.0, elasticNetParam=0)

# Set the pipeline
# Note, the first three HAVE to be in that order, indexer could have been elsewhere
pipeline = Pipeline(stages=[regexTokenizer, cv, idf, indexer, lr])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Fit pipeline
lrModel = pipeline.fit(df2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"Test" model on training data

In [6]:
df3 = lrModel.transform(df2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df3.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [9]:
df3.filter(df3.label == df3.prediction).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

54615

In [11]:
df3.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100000

So we get correct results in ~ 54% cases